In [1]:
import requests # to get the website
import time     # to force our code to wait a little before re-trying to grab a webpage
import re       # to grab the exact element we need
from bs4 import BeautifulSoup # to grab the html elements we need
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


# parameters 
movie = 'free_guy'
pageNum = 16


# create the movie url
page = 'https://www.rottentomatoes.com/m/' + movie + '/reviews/'

# tell python where chrome driver is wrt your current working directory
path = "C:\ProgramData\Anaconda3\chromedriver"
browser = webdriver.Chrome(path)

#define a function to check whether it is the last page or not, for control the loop later
def check_lastpage(element):
    try:
        browser.find_element(By.XPATH, element)
    except NoSuchElementException:
        return False
    return True


# open the URL
browser.get(page)


#creat an empty list
data = []
p = 0


while p < pageNum:
    # if we got the page parse the html 
    page_source = browser.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    
    # get all the review divs
    reviews=soup.findAll('div', {'class':re.compile('review_table_row')})
    for review in reviews:
        
        # initialize to not found
        critic = 'NA'
        rating = 'NA'
        source = 'NA'
        text = 'NA'
        date = 'NA'
        sub = []
        subdata = []
        
        #get critical name information
        critic_find = review.find('a',{'href':re.compile('/critic/')})
        if critic_find:
            critic = critic_find.text.strip()
            
        #get rating information
        rating_fresh = review.find('div',{'class':re.compile('review_icon icon small fresh')})
        rating_rotten = review.find('div',{'class':re.compile('review_icon icon small rotten')})
        if rating_fresh:
            rating = 'fresh'
        elif rating_rotten:
            rating = 'rotten'
        
        #get source information
        source_find = review.find('em')
        if source_find:
            source = source_find.text.strip()
        
        #get text information
        text_find = review.find('div',{'class':re.compile("the_review")})
        if text_find:
            text = text_find.text.strip()
        
        #get date information
        date_find = review.find('div',{'class':re.compile("review-date subtle small")})
        if date_find:
            date = date_find.text.strip()
        
        # make each review's attributes (critic,rating,source,text,date) be tab-separated
        # add all the info to the data link 
        # if some element hasn't been found, the 'NA' string will be added
        sub = [critic,rating,source,text,date]
        subdata = "\t".join(sub)
        data.append(subdata)
    
    #check if it has already turned to the last page or not
    element = '//button [@class="js-prev-next-paging-next btn prev-next-paging__button prev-next-paging__button-right"]'
    if (check_lastpage(element)):
        browser.find_element(By.XPATH, element).click()
        time.sleep(2)
    else:
        break
    p += 1

# close the browser
browser.quit()


# make each article information in a line
str_data = "\n".join(data)

# Save all information in a .txt file
final = open('Ruoxuan_Cao_' + movie + '_' + str(pageNum) + 'pages.txt', 'w', encoding='utf-8')
final.write(str_data)
final.close()

<ipython-input-1-850fbab65d07>:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(path)
